Find all Census Block Groups in Bloomington, IN.

In [75]:
# EXAMPLES: http://andrewgaidus.com/Dot_Density_County_Maps/

import pandas as pd

In [76]:
# Get count of devices residing in each CBG on 07/22/2019
devices_residing07222019 = pd.read_csv('../home_panels/2019-07-22-home-panel-summary.csv')

devices_residing07222019 = devices_residing07222019.rename(columns={'number_devices_residing': 'devices07222019'})
devices_residing07222019 = devices_residing07222019.drop(['date_range_start', 'date_range_end', 'state'], axis=1)

devices_residing07222019.tail()

,census_block_group,devices07222019
220102,560210004015,80
220103,560399676003,82
220104,560319594001,30
220105,560419752002,33
220106,560250011002,107


In [77]:
# Get count of devices residing in each CBG on 02/03/2020
devices_residing02032020 = pd.read_csv('../home_panels/2020-02-03-home-panel-summary.csv')

devices_residing02032020 = devices_residing02032020.rename(columns={'number_devices_residing': 'devices02032020'})
devices_residing02032020 = devices_residing02032020.drop(['date_range_start', 'date_range_end', 'state'], axis=1)

devices_residing02032020.tail()

,census_block_group,devices02032020
220159,560250017002,232
220160,560210010002,62
220161,560430002001,51
220162,560119503002,76
220163,560210015013,80


In [78]:
# Create a list of unique Census Block Groups for Bloomington, IN
bton_cbgs = pd.read_csv('bloomington_cbgs.csv')
bton_cbgs.drop_duplicates('census_block_group', keep=False, inplace=True)
bton_cbgs.count

<bound method DataFrame.count of     census_block_group
0         181050001001
1         181050001002
2         181050001003
3         181050002011
4         181050002012
..                 ...
87        181050015022
88        181050016001
89        181050016002
90        181050016003
91        181199559003

[92 rows x 1 columns]>

In [84]:
# Merge dataframes so we have one dataframe that includes summer and winter population, population change, and percent change
filtered_2019_df = pd.merge(bton_cbgs, devices_residing07222019, on=['census_block_group'])
filtered_2020_df = pd.merge(devices_residing02032020, bton_cbgs, how='inner')

bton_devices_residing = pd.merge(filtered_2019_df, filtered_2020_df, how='left', on='census_block_group')

bton_devices_residing['pop_change'] = bton_devices_residing['devices02032020'] - bton_devices_residing['devices07222019']
bton_devices_residing['pop_pct_change'] = bton_devices_residing['pop_change'] / bton_devices_residing['devices07222019']

bton_devices_residing

,census_block_group,devices07222019,devices02032020,pop_change,pop_pct_change
0,181050001001,65,106,41,0.630769
1,181050001002,70,122,52,0.742857
2,181050001003,52,91,39,0.750000
3,181050002011,4,108,104,26.000000
4,181050002012,4,169,165,41.250000
...,...,...,...,...,...
87,181050015022,56,72,16,0.285714
88,181050016001,94,219,125,1.329787
89,181050016002,61,139,78,1.278689
90,181050016003,11,79,68,6.181818


In [89]:
# Create a dataframe that only contains CBGs where the population increased by more than 100% from summer to winter
bton_transient_df = bton_devices_residing[bton_devices_residing['pop_pct_change'] > 1]
bton_transient_df

In [91]:
bton_transient_df.to_csv('bton_transient_cbgs.csv')

In [90]:
# Create a dataframe that only contains CBGs where the population increased by more than 100% from summer to winter
bton_static_df = bton_devices_residing[bton_devices_residing['pop_pct_change'] <= 1]
bton_static_df

In [92]:
bton_static_df.to_csv('bton_static_cbgs.csv')

I now have two dataframes: one that includes the more transient census block groups (e.g. 'students') and one that is more static (e.g. year-round residents.)